In [1]:
import quandl
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data
import pandas as pd
from iexfinance.stocks import Stock
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import math
from sklearn.preprocessing import scale

In [2]:
#Getting Investor Sentiment data
quandl.ApiConfig.api_key = 'YXv7ndtD_deGW9xeR4s8'
df = quandl.get('AAII/AAII_SENTIMENT')

In [3]:
#changing the date into a column rather than index and then to 'datetime' object type
df.reset_index(level = 0, inplace = True)
df.Date=pd.to_datetime(df.Date)

In [4]:
#dropping missing values
df2 = df.dropna()

In [5]:
#getting USDX data
usdx = pd.read_csv('https://www.quandl.com/api/v3/datasets/CHRIS/ICE_DX1.csv?api_key=YXv7ndtD_deGW9xeR4s8')

In [6]:
#dropping unnecessary values from the investor sentiment data
df2 = df2.drop(['Total','Bullish Average',
                'Bullish Average + St. Dev','Bullish Average - St. Dev'], axis = 1)

In [7]:
#USDX date to 'datetime' object type
usdx['Date'] = pd.to_datetime(usdx['Date'])

In [8]:
#merging USDX to the investor sentiment data
df2 = df2.merge(usdx[['Date','Open','High','Low','Settle','Change']], on='Date')

In [9]:
#calculating change in S&P500 weekly close
ch = [0]
ch1 = [((j-i)/i) for i, j in zip(df2['S&P 500 Weekly Close'][:-1], df2['S&P 500 Weekly Close'][1:])]
for i in ch1:
    ch.append(i)

In [10]:
#adding S&P500 change to the dataset
df2['SP week %change'] = ch

In [11]:
#dropping unncessary data
df3 = df2.drop(['Bullish','Neutral','Bearish'], axis=1)

In [12]:
#getting USTB data
ustb = pd.read_csv('https://www.quandl.com/api/v3/datasets/CHRIS/CME_US1.csv?api_key=YXv7ndtD_deGW9xeR4s8')

In [13]:
#changing USTB date to 'datetime' object type
ustb['Date'] = pd.to_datetime(ustb['Date'])

In [14]:
#changing names of columns
ustb = ustb.rename(index=str, columns={'Last':'USTB last', 'Change':'USTB change', 'Settle':'USTB settle'})

In [15]:
#merging USTB to the investor sentiment dataset
df3 = df3.merge(ustb[['USTB last','USTB change','USTB settle','Date']],on='Date')

In [16]:
#renaming column
df3 = df3.rename(index = str, columns={'Settle':'USDX settle'})

In [17]:
#dropping unnecessary values
df3 = df3.drop(['Change'], axis = 1)

In [18]:
#reading in the S&P500 data that was taken from Yahoo Finance and then used to calculate MACD
sp5 = pd.read_csv('sp5 data.csv')

In [19]:
#changing the S&P500 date to 'datetime' object
sp5.Date = pd.to_datetime(sp5.Date)

In [20]:
#merging the S&P500 Yahoo data to the dataset
df4 = df3.merge(sp5[['SP5 12 day SMA','SP5 26 day SMA','SP5 12 day EMA','SP5 26 day EMA','SP5 MACD', 'Date']], on='Date')

In [21]:
#dropping unnecessary variables
df4 = df4.drop(['Open','High','Low','USTB change','USTB last'], axis = 1)

In [22]:
#calculating the weekly change in USDX futures price
ch2 = [0]
ch3 = [((j-i)/i) for i, j in zip(df4['USDX settle'][:-1], df4['USDX settle'][1:])]
for i in ch3:
    ch2.append(i)

In [23]:
#adding the USDX change to the dataset
df4['USDX %change']=ch2

In [24]:
#calculating the weekly change in USTB futures price
ch4 = [0]
ch5 = [((j-i)/i) for i, j in zip(df4['USTB settle'][:-1], df4['USTB settle'][1:])]
for i in ch5:
    ch4.append(i)

In [25]:
#adding the USTB change to the dataset
df4['USTB %change'] = ch4

In [26]:
#using only the data that occurs after January 1, 2001 for my dataset
df4 = df4[df4['Date']>='2000-01-01']

In [27]:
#calculating the S&P500 volatility
vol = (df4['S&P 500 Weekly High']-df4['S&P 500 Weekly Low'])/df4['S&P 500 Weekly Close']
df4['sp volatility'] = vol

In [28]:
#reading in the DJI data that was taken from Yahoo Finance and then used to calculate MACD
dji = pd.read_csv('dji data.csv')

In [29]:
#dropping unnecessary variables to create the feature set
dfX = df4.drop(['S&P 500 Weekly High','S&P 500 Weekly Low',
                'Bullish 8-Week Mov Avg','Bull-Bear Spread','SP5 12 day SMA','SP5 26 day SMA',
                'SP5 12 day EMA','SP5 26 day EMA','S&P 500 Weekly Close','USDX settle','USTB settle'], axis = 1)


In [30]:
#changing the DJI date to'datetime' object
dji.Date = pd.to_datetime(dji['Date'])

In [31]:
#using only necessary variables
dji2 = dji[['Date','DJI MACD']]

In [32]:
#adding the DJI to the feature set
dfX = dfX.merge(dji2,on='Date')

In [33]:
#creating the class set
dfY = df4[['Date','Bull-Bear Spread']]

In [34]:
#creating the discrete variable for the class
v = pd.cut(dfY['Bull-Bear Spread'], bins=2, labels=np.arange(2), right=False)

In [35]:
#adding the discrete variable to the class set
dfY['cat bull-bear'] = v

/Users/shivpalit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
#merging the feature and class datasets
dfXY = dfX.merge(dfY, on='Date')

In [37]:
#reading in the NASDAQ composite Yahoo Finance data and changing the date to 'datetime' object
ix = pd.read_csv('IXIC data.csv')
ix.Date = pd.to_datetime(ix.Date)

In [38]:
#adding the IXIC MACD to the feature and class dataset
dfXY = dfXY.merge(ix[['IXIC MACD','Date']], on='Date')

In [39]:
#saving to a csv file to use for the analysis
dfXY.to_csv('final_data.csv')